In [9]:
import argparse
import csv
import json
import os

import numpy as np
import pandas as pd

#from benchmark import Benchmark

workload_names = [
    "archeology.json",
    "astronomy.json",
    "biomedical.json" "environment.json",
    "legal.json",
    "wildfire.json",
]


sys_names = {
    'BaselineLLMSystemGPTo3Naive': 'GPT-o3',
    'BaselineLLMSystemGPT4oNaive': 'GPT-4o',
    'BaselineLLMSystemClaude35Naive': 'Claude-3-5',
    'BaselineLLMSystemLlama3_3InstructNaive': 'Llama3_3Instruct',
    'BaselineLLMSystemDeepSeekR1Naive': 'DeepSeek-R1',
    'BaselineLLMSystemQwen2_5CoderNaive': 'Qwen2_5Coder',
    
    'BaselineLLMSystemGPTo3OneShot': 'GPT-o3',
    'BaselineLLMSystemGPT4oOneShot': 'GPT-4o',
    'BaselineLLMSystemClaude35OneShot': 'Claude-3-5',
    'BaselineLLMSystemLlama3_3InstructOneShot': 'Llama3_3Instruct',
    'BaselineLLMSystemDeepSeekR1OneShot': 'DeepSeek-R1',
    'BaselineLLMSystemQwen2_5CoderOneShot': 'Qwen2_5Coder',
    
    'BaselineLLMSystemGPTo3FewShot': 'GPT-o3',
    'BaselineLLMSystemGPT4oFewShot': 'GPT-4o',
    'BaselineLLMSystemClaude35FewShot': 'Claude-3-5',
    'BaselineLLMSystemLlama3_3InstructFewShot': 'Llama3_3Instruct',
    'BaselineLLMSystemDeepSeekR1FewShot': 'DeepSeek-R1',
    'BaselineLLMSystemQwen2_5CoderFewShot': 'Qwen2_5Coder',
}

In [10]:
sut_metrics = {}
for sut_name in sys_names:

    aggregated_result_filepath = "./results/aggregated_results.csv"

    df = pd.read_csv(aggregated_result_filepath)
    metric_aggregation_dict = {}
    for (sut, metric), group in df.groupby(["sut", "metric"]):
        if sut != sut_name:
            continue
        group_dropped_na = group.dropna()
        metric_aggregation_dict[metric] = group["value_mean"].mean()
    # print(f"Aggregated results for {sut_name}:")
    # print(metric_aggregation_dict)
    sut_metrics[sut_name] = metric_aggregation_dict

metrics_df = pd.DataFrame.from_dict(sut_metrics, orient="index")
metrics = ['bleu', 'llm_code_eval', 'f1', 'mean_absolute_error', 'precision', 'recall', 'rouge', 'success', 'runtime']
for m in metrics:
    if m != 'runtime':
        metrics_df[m] = metrics_df[m]*100

display(metrics_df)
ltx_table = metrics_df.to_latex(
    index=True,
    label="tab:metrics",
    caption="Metrics for different systems.",
    float_format="%.2f",
    column_format="l" + "c" * len(metrics_df.columns),
)

for sut_name in sys_names:
    ltx_table = ltx_table.replace(sut_name, "& "+sys_names[sut_name])

print(ltx_table)

,bleu,f1,f1_approximate,llm_code_eval,llm_paraphrase,mean_absolute_error,mean_relative_absolute_error,mean_squared_error,precision,rae_score,recall,rouge,runtime,string_bootstrap,success,total_token_usage_answers,total_token_usage_pipeline
BaselineLLMSystemGPTo3Naive,28.429437,15.887608,0.0,57.666304,0.163889,75676.795000,9.046777e+03,1.145387e+06,20.662577,4523.417257,25.775432,30.463175,7.509467,0.293004,16.897737,15.233077,1761.477116
BaselineLLMSystemGPT4oNaive,19.650903,14.813657,0.0,46.470033,0.055556,1316.280000,8.332667e+03,1.732593e+02,7.492504,4166.333375,9.343542,19.432624,4.791857,0.194326,7.501252,15.502316,1232.749868
BaselineLLMSystemClaude35Naive,18.077269,15.824431,0.0,48.894117,0.083333,NaN,9.999000e+03,NaN,12.112376,4999.500050,16.517024,18.990297,3.750630,0.173574,7.674721,15.177009,1258.724868
BaselineLLMSystemLlama3_3InstructNaive,10.615654,15.905519,0.0,34.559193,0.083333,NaN,9.999000e+03,NaN,11.133245,4999.500050,11.379986,9.444444,5.414066,0.083333,5.789586,15.845586,1112.856481
BaselineLLMSystemQwen2_5CoderNaive,2.243057,14.098076,0.0,39.828377,0.000000,NaN,9.999000e+03,NaN,4.044984,4999.500050,5.319535,1.316872,9.270027,0.000000,1.189844,20.005505,1109.242857
BaselineLLMSystemGPTo3OneShot,36.072754,19.759710,0.0,60.232017,0.188889,918.177500,8.094498e+03,2.678764e+02,26.941128,4047.364212,36.268587,41.738426,10.162886,0.329543,27.216814,15.407115,1455.797222
BaselineLLMSystemGPT4oOneShot,13.024730,16.889501,0.0,44.951074,0.111111,11.500000,7.856497e+03,2.645000e-02,13.633490,3928.309690,17.567980,15.051680,5.348470,0.138889,12.184503,15.532867,1122.065741
BaselineLLMSystemClaude35OneShot,19.509739,14.789980,0.0,57.140101,0.163889,0.020000,6.582845e+10,1.200000e-07,15.022658,3095.138492,22.083579,23.114951,4.755620,0.204293,15.402336,15.279090,1200.506481
BaselineLLMSystemLlama3_3InstructOneShot,5.022368,16.321025,0.0,35.842372,0.027778,282.000000,9.046739e+03,1.466050e+01,9.606793,4523.433231,12.137665,4.094650,6.605255,0.027778,4.250378,15.860534,1072.580952
BaselineLLMSystemQwen2_5CoderOneShot,5.018672,17.304122,0.0,38.315161,0.027778,23.000000,9.522874e+03,5.290000e-02,6.957230,4761.463792,8.451910,4.094650,10.725109,0.027778,5.451516,15.542321,980.078439


\begin{table}
\centering
\caption{Metrics for different systems.}
\label{tab:metrics}
\begin{tabular}{lccccccccccccccccc}
\toprule
{} &  bleu &    f1 &  f1\_approximate &  llm\_code\_eval &  llm\_paraphrase &  mean\_absolute\_error &  mean\_relative\_absolute\_error &  mean\_squared\_error &  precision &  rae\_score &  recall &  rouge &  runtime &  string\_bootstrap &  success &  total\_token\_usage\_answers &  total\_token\_usage\_pipeline \\
\midrule
& GPT-o3              & 28.43 & 15.89 &            0.00 &          57.67 &            0.16 &             75676.79 &                       9046.78 &          1145386.53 &      20.66 &    4523.42 &   25.78 &  30.46 &     7.51 &              0.29 &    16.90 &                      15.23 &                     1761.48 \\
& GPT-4o              & 19.65 & 14.81 &            0.00 &          46.47 &            0.06 &              1316.28 &                       8332.67 &              173.26 &       7.49 &    4166.33 &    9.34 &  19.43 &     4.79 & 

/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/3429854630.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ltx_table = metrics_df.to_latex(


In [11]:
print("Per-domain aggregation:")
# Calculate the weighted mean of the following metrics per domain
domains = [
    "archeology",
    "astronomy",
    "biomedical", "environment",
    "legal",
    "wildfire",
]
metrics = ['success', 'llm_paraphrase', 'rae_score', 'f1']
suts = list(sys_names.keys())
# measures = {}
# for sut in df['sut'].unique():
#     for workload_name in workload_names:
#         df['weighted_metric'] = df['value_support'] + df['value_mean']
#         x = df.groupby(['sut', 'workload'])['value_support'].sum()
#         supports = dict(x)
#         measures = []
#         for key in supports.keys():
#             sut, workload = key
#             sys_domain_measure = df[df['sut'] == sut][df['workload'] == workload]['value_mean'] * df[df['sut'] == sut][df['workload'] == workload]['value_support'] / supports[key]
df['meansupp'] = df['value_mean'] * df['value_support']
results = {}
for domain in domains+['overall', 'runtime']:
    if domain not in ['overall', 'runtime']:
        sut_df = df.query(f'sut in {suts} and workload == "{domain}.json" and metric in {metrics}')
        x = sut_df.groupby(['sut']).sum()['meansupp']
        y = sut_df.groupby(['sut']).sum()['value_support']
        results[domain] = x/y

    elif domain == 'overall':
        sut_df = df.query(f'sut in {suts} and metric in {metrics}')
        x = sut_df.groupby(['sut']).sum()['meansupp']
        y = sut_df.groupby(['sut']).sum()['value_support']
        results[domain] = x/y

    elif domain == 'runtime':
        sut_df = df.query(f'sut in {suts} and metric == "runtime"')
        x = sut_df.groupby(['sut']).sum()['meansupp']
        y = sut_df.groupby(['sut']).sum()['value_support']
        results[domain] = x/y


domain_df = pd.DataFrame(results)*100
domain_df = domain_df.reindex(suts)

display(domain_df)
#ltx_table = domain_df.to_latex(
#    index=True,
#    label="tab:metrics",
#    caption="Metrics for different domains.",
#    float_format="%.2f",
#    column_format="l" + "c" * len(metrics_df.columns),
#)
#for sys_name in sys_names:
#    ltx_table = ltx_table.replace(sys_name, "& "+sys_names[sys_name])

#print(ltx_table)
domain_df.to_markdown(index=True, tablefmt="pipe", floatfmt=".4f")

Per-domain aggregation:


/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/1102534138.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = sut_df.groupby(['sut']).sum()['meansupp']
/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/1102534138.py:28: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = sut_df.groupby(['sut']).sum()['value_support']
/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/1102534138.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which 

,archeology,astronomy,biomedical,environment,legal,wildfire,overall,runtime
sut,,,,,,,,
BaselineLLMSystemGPTo3Naive,17.839651,27407.455658,19.477334,18.675133,9578.367775,37327.716189,12889.544140,660.371373
BaselineLLMSystemGPT4oNaive,15.091342,27403.664806,12.162825,11.258023,4793.572122,52240.728279,14314.913551,401.188829
BaselineLLMSystemClaude35Naive,16.521325,27405.149670,9.866901,12.511544,9578.223662,52242.302409,15746.263898,355.692086
BaselineLLMSystemLlama3_3InstructNaive,14.441303,27406.694428,10.243032,10.348061,9576.617110,52241.643211,15745.123133,429.379902
BaselineLLMSystemDeepSeekR1Naive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BaselineLLMSystemQwen2_5CoderNaive,10.236572,27401.264392,7.712404,7.136039,32256.362098,52238.110683,19198.555065,667.346665
BaselineLLMSystemGPTo3OneShot,23.903794,27415.659472,18.288450,28.477743,9586.911059,22415.596928,10037.056912,906.637232
BaselineLLMSystemGPT4oOneShot,14.255286,13709.413429,9.383845,20.374575,9579.383620,44791.949765,12889.481093,528.040158
BaselineLLMSystemClaude35OneShot,17.065971,13707.497210,9.440771,22.265160,4796.633234,44790.920727,11459.798255,393.675362


'| sut                                      |   archeology |   astronomy |   biomedical |   environment |      legal |   wildfire |    overall |   runtime |\n|:-----------------------------------------|-------------:|------------:|-------------:|--------------:|-----------:|-----------:|-----------:|----------:|\n| BaselineLLMSystemGPTo3Naive              |      17.8397 |  27407.4557 |      19.4773 |       18.6751 |  9578.3678 | 37327.7162 | 12889.5441 |  660.3714 |\n| BaselineLLMSystemGPT4oNaive              |      15.0913 |  27403.6648 |      12.1628 |       11.2580 |  4793.5721 | 52240.7283 | 14314.9136 |  401.1888 |\n| BaselineLLMSystemClaude35Naive           |      16.5213 |  27405.1497 |       9.8669 |       12.5115 |  9578.2237 | 52242.3024 | 15746.2639 |  355.6921 |\n| BaselineLLMSystemLlama3_3InstructNaive   |      14.4413 |  27406.6944 |      10.2430 |       10.3481 |  9576.6171 | 52241.6432 | 15745.1231 |  429.3799 |\n| BaselineLLMSystemDeepSeekR1Naive         |     nan     

In [13]:
print("Per-domain aggregation:")
# Calculate the weighted mean of the following metrics per domain
domains = [
    "archeology",
    "astronomy",
    "biomedical", "environment",
    "legal",
    "wildfire",
]
metrics = ["success", "llm_paraphrase", "rae_score", "f1"]
suts = list(sys_names.keys())
# measures = {}
# for sut in df['sut'].unique():
#     for workload_name in workload_names:
#         df['weighted_metric'] = df['value_support'] + df['value_mean']
#         x = df.groupby(['sut', 'workload'])['value_support'].sum()
#         supports = dict(x)
#         measures = []
#         for key in supports.keys():
#             sut, workload = key
#             sys_domain_measure = df[df['sut'] == sut][df['workload'] == workload]['value_mean'] * df[df['sut'] == sut][df['workload'] == workload]['value_support'] / supports[key]

df.loc[df["metric"] == "rae_score", "value_mean"] = 1 / (1 + df["value_mean"])
df["meansupp"] = df["value_mean"] * df["value_support"]
# change value_mean of rae_score to 1/(1+ value_mean)

results = {}
for domain in domains + ["overall", "runtime"]:
    if domain not in ["overall", "runtime", "rae_score"]:
        sut_df = df.query(
            f'sut in {suts} and workload == "{domain}.json" and metric in {metrics}'
        )
        x = sut_df.groupby(["sut"]).sum()["meansupp"]
        y = sut_df.groupby(["sut"]).sum()["value_support"]
        results[domain] = x / y

    elif domain == "overall":
        sut_df = df.query(f"sut in {suts} and metric in {metrics}")
        x = sut_df.groupby(["sut"]).sum()["meansupp"]
        y = sut_df.groupby(["sut"]).sum()["value_support"]
        results[domain] = x / y

    elif domain == "runtime":
        sut_df = df.query(f'sut in {suts} and metric == "runtime"')
        x = sut_df.groupby(["sut"]).sum()["meansupp"]
        y = sut_df.groupby(["sut"]).sum()["value_support"]
        results[domain] = x / y


domain_df = pd.DataFrame(results) * 100
domain_df = domain_df.reindex(suts)

display(domain_df)
ltx_table = domain_df.to_latex(
    index=True,
    label="tab:metrics",
    caption="Metrics for different domains.",
    float_format="%.2f",
    column_format="l" + "c" * len(metrics_df.columns),
)
for sys_name in sys_names:
    ltx_table = ltx_table.replace(sys_name, "& " + sys_names[sys_name])

print(ltx_table)

Per-domain aggregation:


/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/3207705713.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = sut_df.groupby(["sut"]).sum()["meansupp"]
/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/3207705713.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = sut_df.groupby(["sut"]).sum()["value_support"]
/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/3207705713.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which 

,archeology,astronomy,biomedical,environment,legal,wildfire,overall,runtime
sut,,,,,,,,
BaselineLLMSystemGPTo3Naive,17.839651,12.935932,19.477334,18.675133,9.947009,16.134503,14.934947,660.371373
BaselineLLMSystemGPT4oNaive,15.091342,9.145079,12.162825,11.258023,8.883844,7.147757,10.050060,401.188829
BaselineLLMSystemClaude35Naive,16.521325,10.629944,9.866901,12.511544,9.802897,8.721887,11.071237,355.692086
BaselineLLMSystemLlama3_3InstructNaive,14.441303,12.174702,10.243032,10.348061,8.196345,8.062688,9.930472,429.379902
BaselineLLMSystemDeepSeekR1Naive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BaselineLLMSystemQwen2_5CoderNaive,10.236572,6.744665,7.712404,7.136039,1.524356,4.530160,6.617061,667.346665
BaselineLLMSystemGPTo3OneShot,23.903794,21.139746,18.288450,28.477743,18.490293,25.082778,22.852655,906.637232
BaselineLLMSystemGPT4oOneShot,14.255286,10.578677,9.383845,20.374575,10.962854,19.207521,14.856862,528.040158
BaselineLLMSystemClaude35OneShot,17.065971,10.238990,9.440771,22.265160,11.466487,17.926161,15.476406,393.675362


\begin{table}
\centering
\caption{Metrics for different domains.}
\label{tab:metrics}
\begin{tabular}{lccccccccccccccccc}
\toprule
{} &  archeology &  astronomy &  biomedical &  environment &  legal &  wildfire &  overall &  runtime \\
sut                                      &             &            &             &              &        &           &          &          \\
\midrule
& GPT-o3              &       17.84 &      12.94 &       19.48 &        18.68 &   9.95 &     16.13 &    14.93 &   660.37 \\
& GPT-4o              &       15.09 &       9.15 &       12.16 &        11.26 &   8.88 &      7.15 &    10.05 &   401.19 \\
& Claude-3-5           &       16.52 &      10.63 &        9.87 &        12.51 &   9.80 &      8.72 &    11.07 &   355.69 \\
BaselineLLMSystemLlama3\_3InstructNaive   &       14.44 &      12.17 &       10.24 &        10.35 &   8.20 &      8.06 &     9.93 &   429.38 \\
& DeepSeek-R1         &         NaN &        NaN &         NaN &          NaN &    NaN &       

/state/partition1/slurm_tmp/3299675.0.0/ipykernel_605628/3207705713.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ltx_table = domain_df.to_latex(
